1.1 Jumbo all categories

In [3]:
import requests
import json
import time

def get_categories(url, headers, payload):
    """Fetches product categories from the given URL."""
    try:
        response = requests.request("GET", url, headers=headers, data=payload)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching categories: {e}")
        return []

def get_products_by_category(url, headers, base_payload, category_value):
    """
    Fetches products for a specific category, iterating through pages.
    Returns a list of product slugs for that category.
    """
    category_product_slugs = []
    from_index = 0
    page_size = 200
    
    # Keep track of product IDs to avoid duplicates within a category's pagination
    seen_product_ids = set()

    print(f"--- Fetching products for category: {category_value} ---")

    while True:
        current_payload = base_payload.copy()
        current_payload["from"] = from_index
        current_payload["to"] = from_index + page_size
        
        # Update the selectedFacets for the current category
        current_payload["selectedFacets"] = [
            {
                "key": "category1",
                "value": category_value
            }
        ]

        try:
            response = requests.request("POST", url, headers=headers, data=json.dumps(current_payload))
            response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
            data = response.json()

            products = data.get("products", [])
            if not products:
                print(f"No more products found for {category_value} starting from index {from_index}.")
                break # No more products for this category

            new_products_found = False
            for product in products:
                product_id = product.get("productId")
                slug = product.get("slug")
                if slug and product_id and product_id not in seen_product_ids:
                    category_product_slugs.append(slug)
                    seen_product_ids.add(product_id)
                    new_products_found = True
            
            if not new_products_found:
                print(f"No new unique products found for {category_value} starting from index {from_index}.")
                break # No new unique products, stop pagination

            from_index += page_size
            print(f"Fetched {len(products)} products for {category_value}. Total unique products so far: {len(category_product_slugs)}")
            time.sleep(0.5) # Be polite and avoid hammering the server
        
        except requests.exceptions.RequestException as e:
            print(f"Error fetching products for {category_value} at index {from_index}: {e}")
            break # Break on error to prevent infinite loops

    return category_product_slugs

def main():
    # --- Step 1: Fetch Categories ---
    category_url = "https://assets.jumbo.cl/json/cms/list-category.json"
    category_payload = {}
    category_headers = {
        'sec-ch-ua-platform': '"Windows"',
        'cache-control': 'no-cache',
        'Referer': 'https://www.jumbo.cl/',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Microsoft Edge";v="138"',
        'x-api-key': 'IuimuMneIKJd3tapno2Ag1c1WcAES97j',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0',
        'Content-Type': 'application/json'
    }

    print("Fetching categories...")
    categories = get_categories(category_url, category_headers, category_payload)
    if not categories:
        print("Failed to retrieve categories. Exiting.")
        return

    print(f"Found {len(categories)} categories.")

    # --- Step 2: Iterate through Categories and Fetch Products ---
    product_url = "https://bff.jumbo.cl/catalog/plp"
    # Base payload for product requests, 'from' and 'to' will be updated
    base_product_payload = {
        "store": "jumboclj512",
        "collections": [],
        "fullText": "",
        "brands": [],
        "from": 0,
        "to": 200, # This will be updated dynamically
        "orderBy": "",
        "selectedFacets": [
            {
                "key": "category1",
                "value": "" # This will be set for each category
            }
        ],
        "promotionalCards": True,
        "sponsoredProducts": True
    }
    product_headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'es-419,es;q=0.9,es-ES;q=0.8,en;q=0.7,en-GB;q=0.6,en-US;q=0.5,es-CL;q=0.4',
        'apikey': 'be-reg-groceries-jumbo-catalog-w54byfvkmju5',
        'content-type': 'application/json',
        'origin': 'https://www.jumbo.cl',
        'priority': 'u=1, i',
        'referer': 'https://www.jumbo.cl/',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Microsoft Edge";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0',
        'x-client-platform': 'web',
        'x-client-version': '2.1.21-wl',
        'x-trace-id': '9d1f4028-55a6-4a68-b684-ac84f8a98e62',
        # Note: Cookies can be dynamic and might require a more robust handling if they expire frequently.
        # For this example, we'll use the provided static cookie string.
        'Cookie': 'ConstructorioID_client_id=c3c62e60-fd54-4806-8dc5-c2a86c4d707c; _vwo_uuid_v2=DD2B5E487454009B49780C9843CB4AA96|37ff7aa885967cc797c6c874f1343185; _vwo_uuid=DD2B5E487454009B49780C9843CB4AA96; _vwo_ds=3%241752171694%3A86.42587319%3A%3A; _vis_opt_s=1%7C; _vis_opt_test_cookie=1; _vis_opt_exp_324_combi=2; source=react-app; x-provider-constructor-clientId=c3c62e60-fd54-4806-8dc5-c2a86c4d707c; _gcl_au=1.1.1463900559.1752171695; _ga=GA1.1.759681815.1752171696; _tt_enable_cookie=1; _ttp=01JZTS4T4WMNYT87ZGZA58TKZZ_.tt.1; x-search=constructor|1.4; x-provider-catalog=cenco-search; _hjSessionUser_1246805=eyJpZCI6ImM1ZWE0YTJiLWVmOTEtNWQwZC04ZWEyLTMxMTU0ZTc1NjllYyIsImNyZWF0ZWQiOjE3NTIxNzE3MDg5MzAsImV4aXN0aW5nIjp0cnVlfQ==; ConstructorioID_session_id=3; x-provider-constructor-sessionId=3; _uetsid=a75ed14060c111f09bf555ee24311913; _uetvid=b7e64f405dba11f0a8ffa19e4237975c; ttcsid=1752508552300::kBLtT0QuDXEH7IA0WoI1.3.1752508552300; ttcsid_C8B8REOB3BVP5AF92AP0=1752508552298::HTL4vNwFyyb1ulvqeaFP.3.1752508552612; _hjSession_1246805=eyJpZCI6IjQwMTEyYjQ4LWYyNzAtNGQ5OC1iNzgxLTNmZWM4ZWJhYTU5ZiIsImMiOjE3NTI1MDg1NjIwNDEsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; _ga_263BD8JJ77=GS2.1.s1752508550$o3$g1$t1752508565$j45$l0$h1485177141; ConstructorioID_session={"sessionId":3,"lastTime":1752508565823}; x-provider-constructor-session=%7B%22sessionId%22%3A3%2C%22lastTime%22%3A1752508565823%7D; _vwo_sn=336839%3A2%3A%3A%3A1; _ga_XLPL1HXJ24=GS2.1.s1752508550$o3$g1$t1752508567$j43$l0$h1860073510'
    }

    all_product_slugs = []

    for category in categories:
        # Prepend '/' to the category name as required
        formatted_category = f"/{category}"
        slugs = get_products_by_category(product_url, product_headers, base_product_payload, formatted_category)
        all_product_slugs.extend(slugs)
        print(f"Finished processing category: {category}. Total slugs collected so far: {len(all_product_slugs)}")

    print("\n--- All Product Slugs Collected ---")
    print(f"Total unique product slugs found across all categories: {len(set(all_product_slugs))}") # Use set to count unique slugs
    # print(all_product_slugs) # Uncomment to print the full list of slugs

if __name__ == "__main__":
    main()
    # slugs


Fetching categories...
Found 813 categories.
--- Fetching products for category: /mercado-carnes ---
Fetched 40 products for /mercado-carnes. Total unique products so far: 40
No more products found for /mercado-carnes starting from index 200.
Finished processing category: mercado-carnes. Total slugs collected so far: 40
--- Fetching products for category: /productos-americanos ---
Fetched 200 products for /productos-americanos. Total unique products so far: 200


KeyboardInterrupt: 

1.2 Jumbo - catetegorias principales

In [8]:
import requests
import json
import time
import os # Import the os module for path manipulation

def get_categories(url, headers, payload):
    """
    Fetches product categories from the given URL and extracts URLs
    ONLY from the top-level 'items' under 'acf'.
    """
    try:
        response = requests.request("GET", url, headers=headers, data=payload)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        
        data = response.json()
        
        extracted_urls = []

        # Navigate directly to the 'acf' -> 'items' path
        acf_data = data.get("acf")
        if acf_data and isinstance(acf_data, dict):
            items = acf_data.get("items")
            if items and isinstance(items, list):
                # Iterate only through the top-level items
                for item in items:
                    if isinstance(item, dict):
                        url = item.get("url")
                        if url and isinstance(url, str):
                            extracted_urls.append(url)
        
        # Filter for valid category URLs (non-empty, starts with '/')
        return [u for u in extracted_urls if u and u.startswith('/')]
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching categories: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response for categories: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred while processing categories: {e}")
        return []

def get_products_by_category(url, headers, base_payload, category_value):
    """
    Fetches products for a specific category, iterating through pages.
    Returns a list of product slugs for that category.
    """
    category_product_slugs = []
    from_index = 0
    page_size = 200
    
    # Keep track of product IDs to avoid duplicates within a category's pagination
    seen_product_ids = set()

    print(f"--- Fetching products for category: {category_value} ---")

    while True:
        current_payload = base_payload.copy()
        current_payload["from"] = from_index
        current_payload["to"] = from_index + page_size
        
        # Update the selectedFacets for the current category
        current_payload["selectedFacets"] = [
            {
                "key": "category1",
                "value": category_value
            }
        ]

        try:
            response = requests.request("POST", url, headers=headers, data=json.dumps(current_payload))
            response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
            data = response.json()

            products = data.get("products", [])
            if not products:
                print(f"No more products found for {category_value} starting from index {from_index}.")
                break # No more products for this category

            new_products_found = False
            for product in products:
                product_id = product.get("productId")
                slug = product.get("slug")
                if slug and product_id and product_id not in seen_product_ids:
                    category_product_slugs.append(slug)
                    seen_product_ids.add(product_id)
                    new_products_found = True
            
            if not new_products_found:
                print(f"No new unique products found for {category_value} starting from index {from_index}.")
                break # No new unique products, stop pagination

            from_index += page_size
            print(f"Fetched {len(products)} products for {category_value}. Total unique products so far: {len(category_product_slugs)}")
            time.sleep(0.5) # Be polite and avoid hammering the server
        
        except requests.exceptions.RequestException as e:
            print(f"Error fetching products for {category_value} at index {from_index}: {e}")
            break # Break on error to prevent infinite loops
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response for products in category {category_value}: {e}")
            break
        except Exception as e:
            print(f"An unexpected error occurred while processing products for category {category_value}: {e}")
            break

    return category_product_slugs

def main():
    # --- Step 1: Fetch Categories from the new endpoint (top-level only) ---
    category_url = "https://assets.jumbo.cl/json/cms/page-11732.json"
    category_payload = {}
    category_headers = {
        'accept': '*/*',
        'accept-language': 'es-419,es;q=0.9,es-ES;q=0.8,en;q=0.7,en-GB;q=0.6,en-US;q=0.5,es-CL;q=0.4',
        'cache-control': 'no-cache',
        'content-type': 'application/json',
        'origin': 'https://www.jumbo.cl',
        'pragma': 'no-cache',
        'priority': 'u=1, i',
        'referer': 'https://www.jumbo.cl/',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Microsoft Edge";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0',
        'x-api-key': 'IuimuMneIKJd3tapno2Ag1c1WcAES97j'
    }

    print("Fetching top-level category URLs from the new endpoint...")
    categories = get_categories(category_url, category_headers, category_payload)
    if not categories:
        print("Failed to retrieve categories from the new endpoint. Exiting.")
        return

    print(f"Found {len(categories)} top-level category URLs.")
    # print(categories) # Uncomment to see the list of extracted category URLs

    # --- Step 2: Iterate through Categories and Fetch Products ---
    product_url = "https://bff.jumbo.cl/catalog/plp"
    base_product_payload = {
        "store": "jumboclj512",
        "collections": [],
        "fullText": "",
        "brands": [],
        "from": 0,
        "to": 200,
        "orderBy": "",
        "selectedFacets": [
            {
                "key": "category1",
                "value": ""
            }
        ],
        "promotionalCards": True,
        "sponsoredProducts": True
    }
    product_headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'es-419,es;q=0.9,es-ES;q=0.8,en;q=0.7,en-GB;q=0.6,en-US;q=0.5,es-CL;q=0.4',
        'apikey': 'be-reg-groceries-jumbo-catalog-w54byfvkmju5',
        'content-type': 'application/json',
        'origin': 'https://www.jumbo.cl',
        'priority': 'u=1, i',
        'referer': 'https://www.jumbo.cl/',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Microsoft Edge";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0',
        'x-client-platform': 'web',
        'x-client-version': '2.1.21-wl',
        'x-trace-id': '9d1f4028-55a6-4a68-b684-ac84f8a98e62',
        'Cookie': 'ConstructorioID_client_id=c3c62e60-fd54-4806-8dc5-c2a86c4d707c; _vwo_uuid_v2=DD2B5E487454009B49780C9843CB4AA96|37ff7aa885967cc797c6c874f1343185; _vwo_uuid=DD2B5E487454009B49780C9843CB4AA96; _vwo_ds=3%241752171694%3A86.42587319%3A%3A; _vis_opt_s=1%7C; _vis_opt_test_cookie=1; _vis_opt_exp_324_combi=2; source=react-app; x-provider-constructor-clientId=c3c62e60-fd54-4806-8dc5-c2a86c4d707c; _gcl_au=1.1.1463900559.1752171695; _ga=GA1.1.759681815.1752171696; _tt_enable_cookie=1; _ttp=01JZTS4T4WMNYT87ZGZA58TKZZ_.tt.1; x-search=constructor|1.4; x-provider-catalog=cenco-search; _hjSessionUser_1246805=eyJpZCI6ImM1ZWE0YTJiLWVmOTEtNWQwZC04ZWEyLTMxMTU0ZTc1NjllYyIsImNyZWF0ZWQiOjE3NTIxNzE3MDg5MzAsImV4aXN0aW5nIjp0cnVlfQ==; ConstructorioID_session_id=3; x-provider-constructor-sessionId=3; _uetsid=a75ed14060c111f09bf555ee24311913; _uetvid=b7e64f405dba11f0a8ffa19e4237975c; ttcsid=1752508552300::kBLtT0QuDXEH7IA0WoI1.3.1752508552300; ttcsid_C8B8REOB3BVP5AF92AP0=1752508552298::HTL4vNwFyyb1ulvqeaFP.3.1752508552612; _hjSession_1246805=eyJpZCI6IjQwMTEyYjQ4LWYyNzAtNGQ5OC1iNzgxLTNmZWM4ZWJhYTU5ZiIsImMiOjE3NTI1MDg1NjIwNDEsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; _ga_263BD8JJ77=GS2.1.s1752508550$o3$g1$t1752508565$j45$l0$h1485177141; ConstructorioID_session={"sessionId":3,"lastTime":1752508565823}; x-provider-constructor-session=%7B%22sessionId%22%3A3%2C%22lastTime%22%3A1752508565823%7D; _vwo_sn=336839%3A2%3A%3A%3A1; _ga_XLPL1HXJ24=GS2.1.s1752508550$o3$g1$t1752508567$j43$l0$h1860073510'
    }

    all_product_slugs = []

    # Iterate through the extracted top-level URLs, which now serve as categories
    for category_url_path in categories:
        slugs = get_products_by_category(product_url, product_headers, base_product_payload, category_url_path)
        all_product_slugs.extend(slugs)
        print(f"Finished processing category URL: {category_url_path}. Total slugs collected so far: {len(all_product_slugs)}")

    print("\n--- All Product Slugs Collected ---")
    unique_slugs = sorted(list(set(all_product_slugs))) # Get unique slugs and sort them for consistent output
    print(f"Total unique product slugs found across all categories: {len(unique_slugs)}") 

    # --- Step 3: Save Slugs to a Text File ---
    output_filename = "product_slugs.txt"
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            for slug in unique_slugs:
                f.write(slug + '\n')
        print(f"Successfully saved {len(unique_slugs)} unique product slugs to '{output_filename}'")
        # Get the full path for easier access
        current_directory = os.getcwd()
        full_path = os.path.join(current_directory, output_filename)
        print(f"You can find the file at: {full_path}")
    except IOError as e:
        print(f"Error saving slugs to file '{output_filename}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred while saving the file: {e}")

if __name__ == "__main__":
    main()

Fetching top-level category URLs from the new endpoint...
Found 30 top-level category URLs.
--- Fetching products for category: /marcas-exclusivas ---
Fetched 200 products for /marcas-exclusivas. Total unique products so far: 200
Fetched 200 products for /marcas-exclusivas. Total unique products so far: 400
Fetched 200 products for /marcas-exclusivas. Total unique products so far: 600
Fetched 200 products for /marcas-exclusivas. Total unique products so far: 800
Fetched 36 products for /marcas-exclusivas. Total unique products so far: 836
No more products found for /marcas-exclusivas starting from index 1000.
Finished processing category URL: /marcas-exclusivas. Total slugs collected so far: 836
--- Fetching products for category: /marcas-exclusivas ---
Fetched 200 products for /marcas-exclusivas. Total unique products so far: 200
Fetched 200 products for /marcas-exclusivas. Total unique products so far: 400
Fetched 200 products for /marcas-exclusivas. Total unique products so far: 600

2.1 Unimarc

In [6]:
import requests


def get_categories():
    """Fetches product categories from the given URL."""
    url = "https://bff-unimarc-web.unimarc.cl/bff-api/categories/2"

    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    print(response.text)
    if response.status_code == 200:
        try:
            data = response.json()
            # insder data parameter and inside fqDescriptionSlug (this is the description slug)
            categories = data.get("data", [])
            if not categories:
                print("No categories found in the response.")
                return []
            # Extract the 'fqDescriptionSlug' from each category
            categories = [cat.get("fqDescriptionSlug") for cat in categories if cat.get("fqDescriptionSlug")]
            print(f"Found {len(categories)} categories.")
            return categories
        except json.JSONDecodeError:
            print("Error decoding JSON response")
            return []

categories = get_categories()

{"message":"OK","error":false,"data":[{"id":349,"name":"Carnes","hasChildren":true,"url":"https://unimarc.vtexcommercestable.com.br/carnes","children":[{"id":364,"name":"Vacuno","hasChildren":true,"url":"https://unimarc.vtexcommercestable.com.br/carnes/vacuno","children":[{"id":443,"name":"Molida, bistec y otros","hasChildren":false,"url":"https://unimarc.vtexcommercestable.com.br/carnes/vacuno/molida-bistec-y-otros","children":[],"MetaTagDescription":"¡Carne molida, bistec, pollo y cerdo en Unimarc.cl! Los mejores precios para preparaciones inolvidables .","fq":"C:349/364/443","fqDescription":"Carnes/Vacuno/Molida, bistec y otros","fqDescriptionSlug":"carnes/vacuno/molida-bistec-y-otros"},{"id":444,"name":"Huesos y otros","hasChildren":false,"url":"https://unimarc.vtexcommercestable.com.br/carnes/vacuno/huesos-y-otros","children":[],"MetaTagDescription":"Huesos y otros","fq":"C:349/364/444","fqDescription":"Carnes/Vacuno/Huesos y otros","fqDescriptionSlug":"carnes/vacuno/huesos-y-otro

In [14]:
import requests
import json


# pageProps, dehydratedState, queries, state, data, availableProducts, slug
# obtener el slug de cada producto, iterando por cada categoria


def get_products_by_category(category_slug):
    """Fetches products for a specific category by slug.
    Each category has various pages, so we need to iterate through them.
    until we reach the end of the pages
    """
    page = 1
    all_product_slugs = []
    while True:
        url = f"https://www.unimarc.cl/_next/data/vvbfr39QMXAAZOm7L4o6c/category/{category_slug}.json?page={page}&categories={category_slug}"
        payload = {}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)

        if response.status_code != 200:
            print(f"Failed to fetch products for {category_slug} on page {page}. Status code: {response.status_code}")
            break

        try:
            data = response.json()
            products = data.get("pageProps", {}).get("dehydratedState", {}).get("queries", [])
            
            found_any = False
            for product in products:
                available_products = product.get("state", {}).get("data", {}).get("availableProducts")
                if not available_products:
                    continue
                for available_product in available_products:
                    slug = available_product.get("slug")
                    if slug:
                        all_product_slugs.append(slug)
                print(f"Fetched {len(available_products)} products for {category_slug} on page {page}. Total slugs so far: {len(all_product_slugs)}")
                found_any = True
            if not found_any:
                print(f"No available products found for {category_slug} on page {page}. Ending pagination.")
                break
            page += 1
            # time.sleep(0.05)
        except json.JSONDecodeError:
            print(f"Error decoding JSON response for {category_slug} on page {page}.")
            break
    return all_product_slugs

def main():
    categories = get_categories()
    if not categories:
        print("No categories found. Exiting.")
        return

    all_product_slugs = []

    for category in categories:
        print(f"Fetching products for category: {category}")
        slugs = get_products_by_category(category)
        all_product_slugs.extend(slugs)
        print(f"Finished processing category: {category}. Total slugs collected so far: {len(all_product_slugs)}")

    print("\n--- All Product Slugs Collected ---")
    unique_slugs = sorted(list(set(all_product_slugs)))  # Get unique slugs and sort them for consistent output
    print(f"Total unique product slugs found across all categories: {len(unique_slugs)}")
    # Save slugs to a text file
    output_filename = "unimarc_product_slugs.txt"
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            for slug in unique_slugs:
                f.write(slug + '\n')
        print(f"Successfully saved {len(unique_slugs)} unique product slugs to '{output_filename}'")
    except IOError as e:
        print(f"Error saving slugs to file '{output_filename}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred while saving the file: {e}")
    
if __name__ == "__main__":
    main()



{"message":"OK","error":false,"data":[{"id":349,"name":"Carnes","hasChildren":true,"url":"https://unimarc.vtexcommercestable.com.br/carnes","children":[{"id":364,"name":"Vacuno","hasChildren":true,"url":"https://unimarc.vtexcommercestable.com.br/carnes/vacuno","children":[{"id":443,"name":"Molida, bistec y otros","hasChildren":false,"url":"https://unimarc.vtexcommercestable.com.br/carnes/vacuno/molida-bistec-y-otros","children":[],"MetaTagDescription":"¡Carne molida, bistec, pollo y cerdo en Unimarc.cl! Los mejores precios para preparaciones inolvidables .","fq":"C:349/364/443","fqDescription":"Carnes/Vacuno/Molida, bistec y otros","fqDescriptionSlug":"carnes/vacuno/molida-bistec-y-otros"},{"id":444,"name":"Huesos y otros","hasChildren":false,"url":"https://unimarc.vtexcommercestable.com.br/carnes/vacuno/huesos-y-otros","children":[],"MetaTagDescription":"Huesos y otros","fq":"C:349/364/444","fqDescription":"Carnes/Vacuno/Huesos y otros","fqDescriptionSlug":"carnes/vacuno/huesos-y-otro

Tottus categorias

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")  # Ejecuta Chrome en modo headless
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options)

url = "https://www.tottus.cl/tottus-cl"
driver.get(url)

# Espera a que cargue la página si es necesario (puedes agregar WebDriverWait aquí)

html = driver.page_source
print(html)  # O guarda el HTML en un archivo si prefieres

driver.quit()

<html lang="en-US" dir="ltr"><head><title>Un momento…</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("");background-repeat:no-repeat;backgr

In [13]:
# Instala primero: pip install undetected-chromedriver
import undetected_chromedriver as uc

options = uc.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)
driver.get("https://www.tottus.cl/tottus-cl")
html = driver.page_source
print(html)
driver.quit()

<html lang="en-US" dir="ltr"><head><title>Un momento…</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("");background-repeat:no-repeat;backgr

In [ ]:
# Instala primero: pip install playwright
# Luego ejecuta en terminal: playwright install

# import asyncio
# from playwright.async_api import async_playwright

# async def get_html():
#     async with async_playwright() as p:
#         browser = await p.chromium.launch(headless=True)
#         context = await browser.new_context(
#             user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
#             locale="es-CL"
#         )
#         page = await context.new_page()
#         await page.goto("https://www.tottus.cl/tottus-cl", timeout=60000)
#         await page.wait_for_load_state("networkidle")
#         html = await page.content()
#         print(html)
#         await browser.close()

# # Ejecuta la función asíncrona
# await get_html()

# import asyncio
# from playwright.async_api import async_playwright

# async def get_html():
#     async with async_playwright() as p:
#         browser = await p.chromium.launch(headless=True)
#         context = await browser.new_context(
#             user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
#             locale="es-CL"
#         )
#         page = await context.new_page()
#         await page.goto("https://www.tottus.cl/tottus-cl", timeout=60000)
#         await page.wait_for_load_state("networkidle")
#         html = await page.content()
#         print(html)
#         await browser.close()




# if __name__ == "__main__":
#     asyncio.run(get_html())

import subprocess
import os
import time
import json

json_path = "tottus_next_data.json"

# Elimina el archivo si existe
if os.path.exists(json_path):
    os.remove(json_path)

# Ejecuta el script que genera el archivo JSON
result = subprocess.run(["python", "tottus.py"], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

# Espera a que el archivo aparezca (timeout de 60 segundos)
timeout = 60
start = time.time()
while not os.path.exists(json_path):
    if time.time() - start > timeout:
        raise TimeoutError("El archivo JSON no se generó a tiempo.")
    time.sleep(1)

# Carga el JSON generado
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Ahora puedes procesar el JSON
print(data.keys())

# iterar las categorias mediante el siguiente json path $['props']['pageProps']['page']['containers'][20]['components'][18]['data']['cards']
# hay varios elementos cards, dento de eso necesitamos solo los "link"
cards = data['props']['pageProps']['page']['containers'][20]['components'][18]['data']['cards']
# Ahora puedes filtrar solo los elementos que tengan "link"
links = [card['link'] for card in cards if 'link' in card]
print(links)

JSON guardado en tottus_next_data.json


dict_keys(['props', 'page', 'query', 'buildId', 'isFallback', 'isExperimentalCompile', 'gssp', 'scriptLoader'])
